In [3]:
import polars as pl

In [4]:
dfOutputs = pl.read_csv("../data/outputs.csv")

In [ ]:
import re

def matchDatetime(text: str):
    # pattern = r'(\d{1,2}\.\s?\d{1,2}\.\s?\d{2,4})\s+(\d{1,2}\.\d{2})'

    #  extend to match bad dates like 11.00.09. 2022        06.00
    pattern = r'(\d{1,2}(\.\d{1,2})?\.\s?\d{1,2}\.\s?\d{2,4})\s+(\d{1,2}\.\d{2})'

    matches = re.search(pattern, text)

    d = None
    t = None

    if matches:
        d = matches.group(1)
        t = matches.group(2)

    return d, t

In [23]:
from datetime import datetime

def parseDiffFormats(dateString: str, timeString: str):
    possible_formats = [
        '%d.%m.%Y - %H.%M',
        '%d.%m.%y - %H.%M',
        '%d.00.%m.%Y - %H.%M',
        '%d.00.%m.%y - %H.%M',
    ]

    dt_string = dateString.replace(" ", "") + " - " + timeString

    dt = "00-00-0000 00:00"

    for fmt in possible_formats:
        try:
            dt = datetime.strptime(dt_string, fmt).strftime('%Y-%m-%d %H:%M')
            break
        except ValueError:
            continue

    return dt

    

In [22]:
from datetime import datetime

def extractDatetime(text):
    rawDate, rawTime = matchDatetime(text)

    if rawDate != None & rawTime != None:

        dt = parseDiffFormats(rawDate, rawTime)
        

        #date = datetime.strptime(rawDate, '%d. %m. %Y').strftime('%Y-%m-%d')
        #time = datetime.strptime(rawTime, '%H.%M').strftime('%H:%M')

        # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
        
        

        #print(f"Date: {date}")
        #print(f"Time: {time}")
        return dt
    else:
        #print("No date and time found.")
        return "00-00-0000 00:00"

In [6]:
dfOutputs = dfOutputs.with_columns([
    (pl.col("content").map_elements(lambda x: extractDatetime(x), return_dtype=pl.String).alias('time'))
])

In [7]:
dfOutputs.head()

year,month,file_name,content,time
i64,str,str,str,str
2022,"""September""","""TMP-540""","""Prometne informacije 10. …","""2022-09-10 10:00"""
2022,"""September""","""TMP-226""","""NOVE Prometne informacije …","""2022-09-22 09:00"""
2022,"""September""","""TMP-232""","""Prometne informacije 22. …","""2022-09-22 06:30"""
2022,"""September""","""TMP-554""","""Prometne informacije 09. …","""2022-09-09 16:30"""
2022,"""September""","""TMP-9""","""Prometne informacije 30.…","""2022-09-30 14:30"""


In [8]:
dfOutputs.write_csv("../data/outputs.csv")